In [ ]:
import ast
import networkx as nx
import importlib
import utils
import style
import graph
import copy
importlib.reload(utils)
importlib.reload(style)
importlib.reload(graph)
from utils import ast_to_networkx, node_transform
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from graph import CodeGraph
import xml.dom.minidom
from IPython.display import Image
from style import GeneralStyleParams
import os


In [ ]:
images_folder = 'images/'
filename_test_exp = 'code_samples/test-expansion.py'
filename_test_extr = 'code_samples/test-extraction.py'


# How to load python code into ast

In [ ]:
with open(filename_test_exp, 'r') as file:
    code = file.read()

tree = ast.parse(code)

# Print the AST
print(ast.dump(tree))

visualizing AST using `networkx` library 

In [ ]:
def visualize_ast(filename):
    with open(filename, 'r') as file:
        code = file.read()

    tree = ast.parse(code)
    G = ast_to_networkx(tree)

    node_labels = ({node: node_transform(G, node) for node in G.nodes()})
    pos = graphviz_layout(G, prog="dot")
    pos = {k: (x, 10*y) for k, (x, y) in pos.items()}
    node_size = [(len(node_transform(G, n))+20)*6 for n in G.nodes()]
    nx.draw(G, pos=pos, with_labels=True, node_size=node_size, labels=node_labels, node_shape="s", arrows = True, node_color="none", bbox=dict(facecolor="skyblue", edgecolor='black', boxstyle='round,pad=0.2'), font_size=6)

    plt.show()
G = ast_to_networkx(tree)
visualize_ast(filename_test_exp)

# Better visualization for AST

In [ ]:
def vis_drawio(g: CodeGraph, output_file_name: str = 'images/temp2', params: GeneralStyleParams = GeneralStyleParams()) -> Image:
    """
    to use this function you need to install drawio and cleate an alias for its command line interface.
    for mac:
        drawio="/Applications/draw.io.app/Contents/MacOS/draw.io"
    """
    mxfile = OurGraphToXML(params).graph_to_xml(g)
    xml_str = xml.dom.minidom.parseString(ET.tostring(mxfile)).toprettyxml()
    
    output_file = f'{output_file_name}.png'
    filename    = f'{output_file_name}.drawio'

    with open(filename, 'w') as file:
        file.write(xml_str) 

    bashCommand = f"/Applications/draw.io.app/Contents/MacOS/draw.io -x -f png --scale 2   -o {output_file} {filename}"
    os.system(bashCommand) 

    return Image(output_file)  

# Expanding a function call

In [ ]:
import transforms
importlib.reload(transforms)
importlib.reload(style)
importlib.reload(graph)
from graph import CodeGraph
from style import OurGraphToXML
from transforms import expand_function

G = CodeGraph.from_file(filename_test_exp)

G_new = G.copy()

for node_id, node in G_new.ast_nodes.items():
    if isinstance(node, ast.Call):
        break

G_new = expand_function(G_new, node_id)
print(G.to_source())
print('_____________')
print(G_new.to_source())

params = GeneralStyleParams(y_scaling=1.3)
vis_drawio(G_new, 'images/test-expansion-after', params)


# Extracting a function

In [ ]:
import transforms
importlib.reload(transforms)
from transforms import extract_function

G = CodeGraph.from_file(filename_test_extr)
print(G.to_source())
vis_drawio(G, 'images/test-extraction-before', params)

In [ ]:
importlib.reload(transforms)
from transforms import extract_function
G2 = copy.deepcopy(G)
G2.refresh()

node_id = id(G2.ast_tree.body[0])
G2.ast_nodes[node_id]
G2 = extract_function(G2, node_id, 1, 2)
print(G2.to_source())
vis_drawio(G2, 'images/test-extraction-after', params)

In [ ]:
importlib.reload(graph)
from graph import CodeGraph

GN = CodeGraph.from_file(filename_test_exp)

In [ ]:
output_file_name = 'images/tokens'
mxfile = OurGraphToXML(params).graph_to_xml(GN)
xml_str = xml.dom.minidom.parseString(ET.tostring(mxfile)).toprettyxml()

output_file = f'{output_file_name}.png'
filename    = f'{output_file_name}.drawio'

with open(filename, 'w') as file:
    file.write(xml_str) 

bashCommand = f"/Applications/draw.io.app/Contents/MacOS/draw.io -x -f png --scale 2   -o {output_file} {filename}"
os.system(bashCommand) 
Image(output_file) 

In [ ]:
import metrics
importlib.reload(metrics)
from metrics import calculate_metrics

metrics_before = calculate_metrics(G.to_source())
metrics_after = calculate_metrics(G2.to_source())

print("Metrics Before:")
print(metrics_before)
print('_____________')
print("Metrics After:")
print(metrics_after)

In [ ]:
isinstance(GN, OurGraphWithNewNodes)

In [ ]:
type(GN) is OurGraphWithNewNodes